# A Simple Bitcoin Demo in Python

## 1 Merkle Tree

In [1]:
import hashlib

def sha256double(byts):
    '''Purpose SHA256 twice to the bytes.'''
    return hashlib.sha256(hashlib.sha256(byts).digest()).digest()

In [2]:
class MerkleTree:
    def __init__(self):
        self.txid_list = []
        self.merkle_root = ''

    def make_merkle_tree(self, txid_list, is_sorted=True):
        '''Compute merkle root for the given list of TXIDs.'''
        if len(txid_list) == 0:
            raise ValueError("empty list")

        # Sort TXIDs (strings) in lexicographical order if not ordered
        if is_sorted:
            self.txid_list = txid_list
        else:
            self.txid_list = sorted(txid_list)

        # Compute merkle root row by row
        hashes = [s.encode() for s in self.txid_list]
        while len(hashes) > 1:
            if len(hashes) & 1:  # Number of hashes is odd
                hashes.append(hashes[-1])
            next_hashes = []
            for i in range(0, len(hashes), 2):
                next_hashes.append(hashes[i] + hashes[i+1])
            hashes = [sha256double(s) for s in next_hashes]  # Hashes are bytes, not texts

        self.merkle_root = hashes[0]
        return self.merkle_root  # 32 bytes

## 2 Bitcoin Accounts

In [24]:
import ecdsa
from ecdsa import SigningKey, VerifyingKey, SECP256k1
import base58

class Account:
    def __init__(self, prikey=''):
        '''
        Create a Bitcoin account using the given private key.
        If private key not provided, generate a new one randomly.
        '''
        if prikey:
            self.private_key = prikey
            self.public_key = self.gen_pubkey_from_prikey(prikey)
        else:
            self.private_key, self.public_key = self.gen_keypair()
        
        self.public_key_hash = self.gen_public_key_hash(self.public_key)
        self.address = self.gen_address_from_pkh(self.public_key_hash)

    @staticmethod
    def gen_keypair():
        '''Generate a new pair of keys using SECP256k1 ECDSA.'''
        sk = SigningKey.generate(curve=SECP256k1) # Private key
        vk = sk.verifying_key                     # Public key
        return sk.to_string(), vk.to_string()
        
    @staticmethod
    def gen_pubkey_from_prikey(prikey):
        '''Compute public key from the given private key.'''
        sk = SigningKey.from_string(prikey, curve=SECP256k1)
        vk = sk.verifying_key
        return vk.to_string()

    @staticmethod
    def gen_public_key_hash(pubkey):
        '''Compute public key hash with "double hash".'''
        temp = hashlib.sha256(pubkey).digest()
        h = hashlib.new('ripemd160')
        h.update(temp)
        return h.digest()

    @staticmethod
    def gen_address_from_pkh(pubkeyhash):
        '''Compute base58check encoded address.'''
        
        # Base58check encode
        version_prefix = b'\x00'  # Version prefix of Bitcoin addresses is 0x00
        return base58.b58encode_check(version_prefix + pubkeyhash)

In [25]:
# Generate 100 Bitcoin accounts for testing
test_accounts = [Account() for i in range(100)]

## 3 Signature and Verification

In [26]:
test_message = b"blockchain-ss-2021"

In [27]:
def sign(message, prikey):
    '''Sign a byte message using the private key.'''
    sk = SigningKey.from_string(prikey, curve=SECP256k1)
    return sk.sign(message)
    
def verify(message, signature, pubkey):
    '''
    Verify signature using the public key.
    Returns true if succeed. Otherwise raises errors.
    '''
    vk = VerifyingKey.from_string(pubkey, curve=SECP256k1)
    return vk.verify(signature, message)

In [41]:
fail_flag = False

for account in test_accounts:
    
    test_signature = sign(test_message, account.private_key)
    try:
        verify(test_message, test_signature, account.public_key)
    except ecdsa.BadSignatureError as e:
        print(e)
        fail_flag = True
        
if not fail_flag:
    print("All succeeded.")

All succeeded.


In [38]:
# A bad case
test_signature = sign(test_message, test_accounts[0].private_key)
temp = list(test_signature)
temp[-1] += 1  # Modifying one byte in the signature causes corruption
bad_signature = bytes(temp)

try:
    verify(test_message, bad_signature, test_accounts[0].public_key)
except ecdsa.BadSignatureError as e:
    print(e)

Signature verification failed


In [14]:
bts = b"Hello"
bytes(list(bts))

b'Hello'

## 4 Bitcoin Transactions

## 5 Bitcoin Blocks